In [1]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Mecab
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter
import os
from gensim.models.keyedvectors import Word2VecKeyedVectors
from gensim.models import KeyedVectors
from tensorflow.keras.initializers import Constant


train_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/data/ratings_test.txt')

/opt/conda/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
def load_data(train_data, test_data, num_words=10000):

    tokenizer = Mecab()
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

    # 중복값 제거
    train_data.drop_duplicates(subset=['document'], keep='last')
    test_data.drop_duplicates(subset=['document'], keep='last')

    
    # 결측치 제거
    train_data.dropna()
    test_data.dropna()

    
    # 토큰화
    # 불용어 제거
    X_train = []
    for sentence in train_data['document']:
        tokens = tokenizer.morphs(u'{}'.format(sentence))
        tokens = [token for token in tokens if token not in stopwords]
        X_train.append(tokens)
    
    X_test = []
    for sentence in test_data['document']:
        tokens = tokenizer.morphs(u'{}'.format(sentence))
        tokens = [token for token in tokens if token not in stopwords]
        X_test.append(tokens)

    
    # 사전 구성
                      
    words_list = np.concatenate(X_train).tolist()
    counter = Counter(words_list)
    counter = counter.most_common(num_words-1)
    vocab = ['<UNK>'] + [word for word, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}

                
    # 토큰들을 사전 인덱스로 변환
    def convert_word_to_index(words_list):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in words_list]
        
    X_train = list(map(convert_word_to_index, X_train))
    X_test = list(map(convert_word_to_index, X_test))
    
    
    return X_train, train_data['label'].to_numpy(), X_test, test_data['label'].to_numpy(), word_to_index
        
    
    
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data, 10000)
index_to_word = {index:word for word, index in word_to_index.items()}

`load_data` 함수를 정의합니다.

In [3]:
np.percentile(np.array([len(sentence) for sentence in X_train]), [25, 50, 75, 100])

array([  7.,  12.,  19., 116.])

문장 길이의 중간값은 12단어 정도이며 19단어 정도의 문장이 문장길이 백분위 75인 것을 알 수 있습니다. 그러니 최대 문장 길이는 20 정도로 하면 적당할 것 같습니다.

In [4]:
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=20, padding='pre', truncating='post')
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=20, padding='pre', truncating='post')

최대 문장 길이를 20으로 하여 패딩을 넣어줍니다. 패딩은 앞에 넣고 긴 문장을 자르는 것은 뒤에서부터 자르도록 합니다.

In [5]:
X_val = X_train[120000:]
y_val = y_train[120000:]
X_train = X_train[:120000]
y_train = y_train[:120000]

150000개의 데이터 중 120000개를 훈련용으로, 나머지 30000개를 validation용으로 사용하겠습니다.

In [6]:
model1 = keras.Sequential()
model1.add(keras.layers.Embedding(10000, 100, input_shape=(None,)))
model1.add(keras.layers.LSTM(2048))
model1.add(keras.layers.Dense(10000, activation='relu'))
model1.add(keras.layers.Dense(1, activation='sigmoid')) 

LSTM 레이어를 사용하는 첫 번째 모델을 설계합니다.

In [7]:
model2 = keras.Sequential()
model2.add(keras.layers.Embedding(10000, 100, input_length=20))
model2.add(keras.layers.GlobalAveragePooling1D())
model2.add(keras.layers.Dense(10000, activation='relu'))
model2.add(keras.layers.Dense(1, activation='sigmoid'))

`GlobalAveragePooling1D`만을 사용하는 모델도 시도해 봅니다.

In [8]:
model3 = keras.Sequential()
model3.add(keras.layers.Embedding(10000, 100, input_shape=(None,)))
model3.add(keras.layers.Conv1D(16, 2, activation='relu'))
model3.add(keras.layers.MaxPooling1D(5))
model3.add(keras.layers.Conv1D(16, 2, activation='relu'))
model3.add(keras.layers.GlobalMaxPooling1D())
model3.add(keras.layers.Dense(10000, activation='relu'))
model3.add(keras.layers.Dense(1, activation='sigmoid')) 

세번째 모델로는 비교를 위해 CNN을 시도해 봅니다.

In [9]:
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5)
model1.evaluate(X_test, y_test)

Epoch 1/5
3750/3750 [==============================] - 405s 107ms/step - loss: 0.4622 - accuracy: 0.7779 - val_loss: 0.3919 - val_accuracy: 0.8236
Epoch 2/5
3750/3750 [==============================] - 405s 108ms/step - loss: 0.3564 - accuracy: 0.8430 - val_loss: 0.3459 - val_accuracy: 0.8501
Epoch 3/5
3750/3750 [==============================] - 405s 108ms/step - loss: 0.2973 - accuracy: 0.8729 - val_loss: 0.3486 - val_accuracy: 0.8498
Epoch 4/5
3750/3750 [==============================] - 406s 108ms/step - loss: 0.2658 - accuracy: 0.8881 - val_loss: 0.3471 - val_accuracy: 0.8500
Epoch 5/5
1563/1563 [==============================] - 60s 38ms/step - loss: 0.3768 - accuracy: 0.8418


[0.37677720189094543, 0.841759979724884]

In [10]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5)
model2.evaluate(X_test, y_test)

Epoch 1/5
3750/3750 [==============================] - 29s 8ms/step - loss: 0.4438 - accuracy: 0.7830 - val_loss: 0.3727 - val_accuracy: 0.8348
Epoch 2/5
3750/3750 [==============================] - 28s 8ms/step - loss: 0.3338 - accuracy: 0.8528 - val_loss: 0.3683 - val_accuracy: 0.8364
Epoch 3/5
3750/3750 [==============================] - 28s 8ms/step - loss: 0.3046 - accuracy: 0.8655 - val_loss: 0.3829 - val_accuracy: 0.8381
Epoch 4/5
3750/3750 [==============================] - 28s 8ms/step - loss: 0.2780 - accuracy: 0.8786 - val_loss: 0.3953 - val_accuracy: 0.8374
Epoch 5/5
1563/1563 [==============================] - 3s 2ms/step - loss: 0.4160 - accuracy: 0.8266


[0.4159991145133972, 0.8265799880027771]

In [11]:
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5)
model3.evaluate(X_test, y_test)

Epoch 1/5
3750/3750 [==============================] - 37s 9ms/step - loss: 0.5522 - accuracy: 0.6933 - val_loss: 0.4877 - val_accuracy: 0.7528
Epoch 2/5
3750/3750 [==============================] - 31s 8ms/step - loss: 0.4290 - accuracy: 0.7862 - val_loss: 0.4757 - val_accuracy: 0.7598
Epoch 3/5
3750/3750 [==============================] - 32s 9ms/step - loss: 0.3624 - accuracy: 0.8224 - val_loss: 0.5002 - val_accuracy: 0.7547
Epoch 4/5
3750/3750 [==============================] - 31s 8ms/step - loss: 0.3105 - accuracy: 0.8478 - val_loss: 0.5548 - val_accuracy: 0.7495
Epoch 5/5
1563/1563 [==============================] - 4s 2ms/step - loss: 0.5950 - accuracy: 0.7414


[0.595029354095459, 0.741379976272583]

모델들을 훈련시켜 주고 `evaluate` 함수로 평가합니다.

In [12]:
word2vec_file_path = os.getenv('HOME')+'/aiffel/sentiment_classification/data/word2vec.txt'
f = open(word2vec_file_path, 'w')
f.write('{} {}\n'.format(9999, 100))

vectors = model1.get_weights()[0]
for i in range(1, 10000):
    f.write('{} {}\n'.format(index_to_word[i], ' '.join(map(str, list(vectors[i, :])))))
f.close()

파일에 모델의 임베딩 벡터를 입력합니다.

In [13]:
word_vectors = Word2VecKeyedVectors.load_word2vec_format(word2vec_file_path, binary=False)
word_vectors.similar_by_word("사랑")

[('비극', 0.9299500584602356),
 ('영리', 0.9295225143432617),
 ('핫', 0.9285176992416382),
 ('차분', 0.9196135401725769),
 ('탄탄', 0.9184383749961853),
 ('쭈욱', 0.9180181622505188),
 ('상큼', 0.9170023798942566),
 ('아름다워서', 0.9167653322219849),
 ('논스톱', 0.9143279790878296),
 ('엘', 0.9127060770988464)]

`similar_by_word`를 호출해보니 사랑과 별로 관련 없는 단어도 많이 나오는군요.

In [14]:
embedding_matrix = np.random.rand(10000, 100)

for i in range(1,10000):
    if index_to_word[i] in word_vectors:
        embedding_matrix[i] = word_vectors[index_to_word[i]]


w2v_model = keras.Sequential()
w2v_model.add(keras.layers.Embedding(10000, 
                                 100, 
                                 embeddings_initializer=Constant(embedding_matrix),
                                 input_length=20, 
                                 trainable=True, input_shape=(None,)))
w2v_model.add(keras.layers.Conv1D(16, 2, activation='relu'))
w2v_model.add(keras.layers.MaxPooling1D(5))
w2v_model.add(keras.layers.Conv1D(16, 2, activation='relu'))
w2v_model.add(keras.layers.GlobalMaxPooling1D())
w2v_model.add(keras.layers.Dense(8, activation='relu'))
w2v_model.add(keras.layers.Dense(1, activation='sigmoid'))

w2v_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']) 

w2v_model.fit(X_train,y_train, epochs=5, batch_size=512, validation_data=(X_val, y_val), verbose=1)

Epoch 1/5
235/235 [==============================] - 9s 25ms/step - loss: 0.5626 - accuracy: 0.7040 - val_loss: 0.4905 - val_accuracy: 0.7548
Epoch 2/5
235/235 [==============================] - 3s 13ms/step - loss: 0.4343 - accuracy: 0.7824 - val_loss: 0.4890 - val_accuracy: 0.7526
Epoch 3/5
235/235 [==============================] - 3s 12ms/step - loss: 0.4029 - accuracy: 0.8007 - val_loss: 0.4903 - val_accuracy: 0.7538
Epoch 4/5
235/235 [==============================] - 3s 13ms/step - loss: 0.3769 - accuracy: 0.8164 - val_loss: 0.5075 - val_accuracy: 0.7496
Epoch 5/5
235/235 [==============================] - 3s 13ms/step - loss: 0.3493 - accuracy: 0.8300 - val_loss: 0.5190 - val_accuracy: 0.7523


In [15]:
w2v_model.evaluate(X_test, y_test)

1563/1563 [==============================] - 4s 2ms/step - loss: 0.5202 - accuracy: 0.7531


[0.5202035307884216, 0.7530999779701233]

In [16]:
# file1 = open(os.getenv('HOME')+'/aiffel/sentiment_classification/data/ko.tsv', 'r')
# lines = file1.read().replace('\n', '')
# lines = re.sub("[0-9]+\t", "", lines)
# lines = re.sub("\[ +","", lines)
# lines = re.sub("\[","", lines)
# lines = re.sub(" +"," ", lines)
# lines = re.sub("\t", " ", lines)
# lines = re.sub("\]", "\n", lines)

# file2 = open(os.getenv('HOME')+'/aiffel/sentiment_classification/data/refined_ko.txt', 'w+')
# file2.write('30185 200\n')
# file2.write(lines)
# file2.close()
# file1.close()

노드에 링크된 한국어 Word2Vec이 bin 파일은 현재 클라우드 gensim과 호환이 안 되고 tsv 파일은 `load_word2vec_format`에서 필요로 하는 형태로 되어있지 않아 전처리를 합니다. 한 번만 하면 파일은 저장되기에 주석처리했습니다

In [17]:
word2vec_path = os.getenv('HOME')+'/aiffel/sentiment_classification/data/refined_ko.txt'
word2vec = KeyedVectors.load_word2vec_format(word2vec_path, binary=False, limit=1000000)
print(word2vec.similar_by_word("사랑"))

embedding_matrix = np.random.rand(10000, 200)

for i in range(1, 10000):
    if index_to_word[i] in word2vec:
        embedding_matrix[i] = word2vec[index_to_word[i]]

[('슬픔', 0.7216663360595703), ('행복', 0.6759076714515686), ('절망', 0.6468985676765442), ('기쁨', 0.6458414196968079), ('이별', 0.6334798336029053), ('추억', 0.6320937871932983), ('인생', 0.6216273903846741), ('애정', 0.6206069588661194), ('연인', 0.6186063289642334), ('유혹', 0.5965287685394287)]


한국어 Word2Vec에서는 좀 더 '사랑'과 비슷한 단어들이 나왔습니다.

In [18]:
pretrained_model = keras.Sequential()
pretrained_model.add(keras.layers.Embedding(10000, 
                                 200, 
                                 embeddings_initializer=Constant(embedding_matrix),
                                 input_length=20,
                                 trainable=True))
pretrained_model.add(keras.layers.LSTM(2048))
pretrained_model.add(keras.layers.Dense(10000, activation='relu'))
pretrained_model.add(keras.layers.Dense(1, activation='sigmoid')) 

pretrained_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']) 

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=1, min_delta=0.005, min_lr=1e-5)
history = pretrained_model.fit(X_train,
                    y_train,
                    epochs=5,
                    batch_size=512,
                    validation_data=(X_val, y_val),
                    verbose=1, callbacks=[reduce_lr])

pretrained_model.evaluate(X_test, y_test)

Epoch 1/5
235/235 [==============================] - 164s 691ms/step - loss: 0.6502 - accuracy: 0.6693 - val_loss: 0.4154 - val_accuracy: 0.8059
Epoch 2/5
235/235 [==============================] - 165s 701ms/step - loss: 0.3585 - accuracy: 0.8413 - val_loss: 0.3383 - val_accuracy: 0.8525
Epoch 3/5
235/235 [==============================] - 164s 699ms/step - loss: 0.2979 - accuracy: 0.8725 - val_loss: 0.3348 - val_accuracy: 0.8541
Epoch 4/5
235/235 [==============================] - 163s 695ms/step - loss: 0.2440 - accuracy: 0.8985 - val_loss: 0.3385 - val_accuracy: 0.8619
Epoch 5/5
1563/1563 [==============================] - 61s 39ms/step - loss: 0.3481 - accuracy: 0.8571


[0.3480691909790039, 0.8570799827575684]

Word2Vec을 사용해 훈련시키니 금방 accuracy가 85% 이상이 나왔습니다

# 회고

**어려웠던 점**: Word2Vec 벡터 파일 전처리가 제일 삽질이었습니다...

**알아낸 점**: 사전을 이용하는 자연어처리 과정이 이해된 것 같습니다.

**아직 모호한 점**: 벡터 파일을 이용해도 정확도가 딱히 높아지지는 않는 이유를 모르겠습니다...

**평가 지표를 맞추기 위해 시도한 것들**: `gensim`을 사용해 비슷한 단어 찾기를 해 보았고 한국어 Word2Vec을 사용해 보았습니다.

**다짐**: 자연어처리를 더 열심히 공부해봐야겠습니다.